# Smart 

## import libraly

In [3]:
import gdown
import pandas as pd

## Dowload dataset from google disk

In [10]:
# URL to the file on Google Drive
url_rents = 'https://drive.google.com/uc?id=1oc_RJRsQEiJutVdWjlRdbJ773YpE9h6x'
output_rents = 'snp_dld_2024_rents.csv'
url_trans = 'https://drive.google.com/uc?id=1liNykIOnfR5KRR4MXJISCZCKJYFnXQC1'
output_trans = 'snp_dld_2024_transactions.csv'
# Download the file
gdown.download(url_rents, output_rents, quiet=False)
gdown.download(url_trans, output_trans, quiet=False)

# Load data into DataFrame
snp_dld_2024_rents = pd.read_csv(output_rents)
snp_dld_2024_transactions = pd.read_csv(output_trans)
print(snp_dld_2024_rents.shape)
print(snp_dld_2024_transactions.shape)

Downloading...
From (original): https://drive.google.com/uc?id=1oc_RJRsQEiJutVdWjlRdbJ773YpE9h6x
From (redirected): https://drive.google.com/uc?id=1oc_RJRsQEiJutVdWjlRdbJ773YpE9h6x&confirm=t&uuid=947b04bc-159f-4ac8-b21c-e9e7c1920b8d
To: /workspaces/SB-Case-Study-Materials/snp_dld_2024_rents.csv
100%|██████████| 278M/278M [00:05<00:00, 53.0MB/s] 
Downloading...
From: https://drive.google.com/uc?id=1liNykIOnfR5KRR4MXJISCZCKJYFnXQC1
To: /workspaces/SB-Case-Study-Materials/snp_dld_2024_transactions.csv
100%|██████████| 70.1M/70.1M [00:05<00:00, 11.7MB/s]
/tmp/ipykernel_2028/3217814367.py:11: DtypeWarning: Columns (35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  snp_dld_2024_rents = pd.read_csv(output_rents)


(681447, 43)
(162806, 46)
